In [1]:
Link = 'https://github.com/natsunoyuki/blog_posts/blob/main/data_science/Bayesian%20Optimization%20of%20Model%20Hyperparameters.ipynb'

In [2]:
pip install niapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 181 kB 15.0 MB/s 


In [3]:
pip install sklearn 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=af65c582e942dcc274512588bab3e21bf7bf9b30bd06556f1e2424275707ed24
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built sklearn


In [4]:
pip install sklearn-nature-inspired-algorithms

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 176 kB 23.1 MB/s 
     |████████████████████████████████| 17.1 MB 34 kB/s 
     |████████████████████████████████| 9.4 MB 39.9 MB/s 
     |████████████████████████████████| 12.2 MB 59.6 MB/s 
     |████████████████████████████████| 295 kB 64.3 MB/s 
     |████████████████████████████████| 965 kB 72.0 MB/s 
     |████████████████████████████████| 16.9 MB 641 kB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
  Attempting uninstall: 

In [5]:
pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [60]:
SEED = 1412
Test_Ratio = 0.2
import pandas as pd

In [61]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [62]:
import os
os.chdir('/content/gdrive/MyDrive/final result bayesian')

In [63]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

def Make_Db():
  X,y = make_classification(n_samples=10000,n_features=5,n_informative=2,n_classes=2,n_clusters_per_class=1,flip_y=float(f'0.{SEED}'),shuffle=False,random_state=SEED)
  #95 -> 4.5%
  #90 -> 14%
  #89 -> 14.12%
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = Test_Ratio, random_state = SEED)

  Train = pd.DataFrame(X_train)
  Train['Answers'] = y_train
  Test = pd.DataFrame(X_test)
  Test['Answers'] = y_test

  Train.to_csv('Train.csv')
  Test.to_csv('Test.csv')


In [64]:
if 'Train.csv' not in os.listdir() or 'Test.csv' not in os.listdir(): 
  Make_Db()
  if 'Final Result.json' in os.listdir():
    os.remove('Final Result.json')


In [65]:
Train = pd.read_csv('Train.csv')
Test = pd.read_csv('Test.csv')
col_lst = list(Test.columns)
col_lst.remove('Answers')
col_lst = [i for i in col_lst if not i.startswith('Unnamed')]
X_train = Train[col_lst]
X_test = Test[col_lst]
y_train = Train['Answers']
y_test = Test['Answers']

In [66]:
len(Test),len(Train)

(2000, 8000)

In [67]:
import json
import numpy as np 
import niapy
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
from sklearn.ensemble import RandomForestClassifier


In [68]:
MyFinalResultDict = dict()

try:
  with open('Final Result.json','r') as dict_dump:
      MyFinalResultDict = json.load(dict_dump)
except:
  pass 

def Save_Final_Dict():
  with open('Final Result.json','w') as dict_dump:
      json.dump(MyFinalResultDict,dict_dump)
  print('Just dumped it!')

SCORING FUNCTIONS (6)

CUCKOO SEARCH

In [69]:
from niapy.algorithms.basic import CuckooSearch

CucKoo_pbounds = {'population_size':(10,100),'pa':(0.1,1.0) ,'n_estimator':(10,1000),'criterion':(0,1),'max_feature':(0,1)}

Cuckoo_dict = dict()

def Save_cuckoo_dict():
    with open('Cuckoo_results.json','w') as dict_dump:
        json.dump(Cuckoo_dict,dict_dump)
    MyFinalResultDict['Cuckoo Top 10'] = sorted(Cuckoo_dict.items())[-10:]


def Score_Cuckoo(population_size,pa,n_estimator,criterion,max_feature):
    
    population_size = int(population_size)
    n_estimator = int(n_estimator)
    if criterion < 0.5:
        criterion = 'gini'
    else:
        criterion = 'entropy'
    if max_feature < 0.34:
        max_feature = 'sqrt'
    elif max_feature < 0.67:
        max_feature = 'log2'
    else:
        max_feature = None

    combo_str = f'{population_size}_{pa}_{n_estimator}_{criterion}_{max_feature}'

    if combo_str in Cuckoo_dict:
        return Cuckoo_dict[combo_str]

    Algo = CuckooSearch()
    Algo.set_parameters(population_size=population_size,pa=pa,seed=SEED)

    nia_mdl = NatureInspiredSearchCV(
        estimator=RandomForestClassifier(n_estimators=n_estimator,criterion=criterion,max_features=max_feature),
        param_grid={},
        algorithm=Algo,
        runs=1
    )
    nia_mdl.fit(X_train,y_train)
    val = nia_mdl.score(X_test,y_test)
    Cuckoo_dict[combo_str] = val 

    return val


CUCKOO SEARCH

FIRE FLY 

In [70]:
from niapy.algorithms.basic import FireflyAlgorithm

FireFly_pbounds = {'population_size':(10,100),'alpha':(0.1,1.0),'beta0':(0.01,100),'gamma':(0.1,1.0),'theta':(0.1,1.0),'n_estimator':(10,1000),'criterion':(0,1),'max_feature':(0,1)}

Firefly_dict = dict()

def Save_firefly_dict():
    with open('Firefly_results.json','w') as dict_dump:
        json.dump(Firefly_dict,dict_dump)
    MyFinalResultDict['FireFly Top 10'] = sorted(Firefly_dict.items())[-10:]

def Score_Firefly(population_size,alpha,beta0,gamma,theta,n_estimator,criterion,max_feature):
    
    population_size = int(population_size)
    n_estimator = int(n_estimator)
    if criterion < 0.5:
        criterion = 'gini'
    else:
        criterion = 'entropy'
    if max_feature < 0.34:
        max_feature = 'sqrt'
    elif max_feature < 0.67:
        max_feature = 'log2'
    else:
        max_feature = None

    combo_str = f'{population_size}_{alpha}_{beta0}_{gamma}_{theta}_{n_estimator}_{criterion}_{max_feature}'

    if combo_str in Firefly_dict:
        return Firefly_dict[combo_str]

    Algo = FireflyAlgorithm()
    Algo.set_parameters(population_size=population_size,alpha=alpha,beta0=beta0,gamma=gamma,theta=theta,seed=SEED)

    nia_mdl = NatureInspiredSearchCV(
        estimator=RandomForestClassifier(n_estimators=n_estimator,criterion=criterion,max_features=max_feature),         
        param_grid={},
        algorithm=Algo,
        runs=1
    )
    nia_mdl.fit(X_train,y_train)
    val = nia_mdl.score(X_test,y_test)
    Firefly_dict[combo_str] = val 

    return val


FIRE FLY 

BAT

In [71]:
from niapy.algorithms.basic import BatAlgorithm

BAT_pbounds = {'population_size':(10,100),'loudness':(0.1,1.0),'pulse_rate':(0.1,1.0),'gamma':(0.1,1.0),'alpha':(0.1,1.0),'min_frequency':(0,10),'max_frequency':(10,100),'n_estimator':(10,1000),'criterion':(0,1),'max_feature':(0,1)}

Bat_dict = dict()

def Save_bat_dict():
    with open('Bat_results.json','w') as dict_dump:
        json.dump(Bat_dict,dict_dump)
    MyFinalResultDict['Bat Top 10'] = sorted(Bat_dict.items())[-10:]


def Score_Bat(population_size,loudness,pulse_rate,gamma,alpha,min_frequency,max_frequency,n_estimator,criterion,max_feature):
    
    population_size = int(population_size)
    n_estimator = int(n_estimator)
    if criterion < 0.5:
        criterion = 'gini'
    else:
        criterion = 'entropy'
    if max_feature < 0.34:
        max_feature = 'sqrt'
    elif max_feature < 0.67:
        max_feature = 'log2'
    else:
        max_feature = None

    combo_str = f'{population_size}_{loudness}_{pulse_rate}_{gamma}_{alpha}_{min_frequency}_{max_frequency}_{n_estimator}_{criterion}_{max_feature}'

    if combo_str in Bat_dict:
        return Bat_dict[combo_str]

    Algo = BatAlgorithm()
    Algo.set_parameters(population_size=population_size,loudness=loudness,pulse_rate=pulse_rate,alpha=alpha,gamma=gamma,min_frequency=min_frequency,max_frequency=max_frequency,seed=SEED)

    nia_mdl = NatureInspiredSearchCV(
        estimator=RandomForestClassifier(n_estimators=n_estimator,criterion=criterion,max_features=max_feature),         
        param_grid={},
        algorithm=Algo,
        runs=1
    )
    nia_mdl.fit(X_train,y_train)
    val = nia_mdl.score(X_test,y_test)
    Bat_dict[combo_str] = val 

    return val


BAT

SELF ADAPTIVE BAT

In [72]:
from niapy.algorithms.modified import AdaptiveBatAlgorithm

SABA_pbounds = {'population_size':(10,100),'loudness':(0.1,1.0),'pulse_rate':(0.1,1.0),'epsilon':(0,0.1),'alpha':(0.1,1.0),'min_frequency':(0,10),'max_frequency':(10,100),'n_estimator':(10,1000),'criterion':(0,1),'max_feature':(0,1)}

SBA_dict = dict()

def Save_sba_dict():
    with open('SBA_results.json','w') as dict_dump:
        json.dump(SBA_dict,dict_dump)
    MyFinalResultDict['SBA Top 10'] = sorted(SBA_dict.items())[-10:]

def Score_SBA(population_size,loudness,pulse_rate,epsilon,alpha,min_frequency,max_frequency,n_estimator,criterion,max_feature):
    
    population_size = int(population_size)
    n_estimator = int(n_estimator)
    if criterion < 0.5:
        criterion = 'gini'
    else:
        criterion = 'entropy'
    if max_feature < 0.34:
        max_feature = 'sqrt'
    elif max_feature < 0.67:
        max_feature = 'log2'
    else:
        max_feature = None

    combo_str = f'{population_size}_{loudness}_{pulse_rate}_{epsilon}_{alpha}_{min_frequency}_{max_frequency}_{n_estimator}_{criterion}_{max_feature}'

    if combo_str in SBA_dict:
        return SBA_dict[combo_str]

    Algo = AdaptiveBatAlgorithm()
    Algo.set_parameters(population_size=population_size,loudness=loudness,pulse_rate=pulse_rate,alpha=alpha,epsilon=epsilon,min_frequency=min_frequency,max_frequency=max_frequency,seed=SEED)

    nia_mdl = NatureInspiredSearchCV(
        estimator=RandomForestClassifier(n_estimators=n_estimator,criterion=criterion,max_features=max_feature),         
        param_grid={},
        algorithm=Algo,
        runs=1
    )
    nia_mdl.fit(X_train,y_train)
    val = nia_mdl.score(X_test,y_test)
    SBA_dict[combo_str] = val 

    return val


SELF ADAPTIVE BAT

PARTICLE SWARM

In [73]:
from niapy.algorithms.basic import ParticleSwarmAlgorithm

PSA_pbounds = {'population_size':(10,100),'c1':(0,4),'c2':(0,4),'w':(0,1),'min_velocity':(-10,0),'max_velocity':(0,10),'repair':(0,5),'n_estimator':(10,1000),'criterion':(0,1),'max_feature':(0,1)}

PSA_dict = dict()

def Save_psa_dict():
    with open('PSA_results.json','w') as dict_dump:
        json.dump(PSA_dict,dict_dump)
    MyFinalResultDict['PSA Top 10'] = sorted(PSA_dict.items())[-10:]

def Score_PSA(population_size,c1,c2,w,min_velocity,max_velocity,repair,n_estimator,criterion,max_feature):

    population_size = int(population_size)
    n_estimator = int(n_estimator)
    if criterion < 0.5:
        criterion = 'gini'
    else:
        criterion = 'entropy'
    if max_feature < 0.34:
        max_feature = 'sqrt'
    elif max_feature < 0.67:
        max_feature = 'log2'
    else:
        max_feature = None
    #limit, limit_inverse, wang, rand, reflect
    repair_name = ''
    if repair < 1.0:
        repair = niapy.util.repair.limit
        repair_name = 'limit'
    elif repair < 2.0:
        repair = niapy.util.repair.limit_inverse
        repair_name = 'limit_inverse'
    elif repair < 3.0:
        repair = niapy.util.repair.wang
        repair_name = 'wang'
    elif repair < 4.0:
        repair = niapy.util.repair.rand
        repair_name = 'rand'
    else:
        repair = niapy.util.repair.reflect
        repair_name = 'reflect'


    combo_str = f'{population_size}_{c1}_{c2}_{w}_{min_velocity}_{max_velocity}_{repair_name}_{n_estimator}_{criterion}_{max_feature}'

    if combo_str in PSA_dict:
        return PSA_dict[combo_str]

    Algo = ParticleSwarmAlgorithm()
    Algo.set_parameters(population_size=population_size,c1=c1,c2=c2,w=w,min_velocity=min_velocity,max_velocity=max_velocity,repair=repair,seed=SEED) 

    nia_mdl = NatureInspiredSearchCV(
        estimator=RandomForestClassifier(n_estimators=n_estimator,criterion=criterion,max_features=max_feature),         
        param_grid={},
        algorithm=Algo,
        runs=1,
    )
    nia_mdl.fit(X_train,y_train)
    val = nia_mdl.score(X_test,y_test)
    PSA_dict[combo_str] = val 

    return val
Link = 'https://www.researchgate.net/profile/Mohamed_Mourad_Lafifi/post/Which-is-the-best-swarm-size-in-PSO/attachment/5b5b6f85b53d2f89289c14e1/AS%3A653084896288769%401532718981208/download/Good+Parameters+for+Particle+Swarm+Optimization.pdf'

PARTICLE SWARM

CamelAlgorithm

In [74]:
from niapy.algorithms.basic import CamelAlgorithm

camel_bounds = {'population_size':(10,100), 'burden_factor':(0.1,1.0), 'death_rate':(0.1,1.0), 'visibility':(0.1,1.0),'supply_init':(1,1000),'endurance_init':(0,1000), 'min_temperature':(-100,0), 'max_temperature':(1,100),'n_estimator':(10,1000),'criterion':(0,1),'max_feature':(0,1)}

camel_dict = dict()

def Save_camel_dict():
    with open('Camel_results.json','w') as dict_dump:
        json.dump(camel_dict,dict_dump)
    MyFinalResultDict['Camel Top 10'] = sorted(camel_dict.items())[-10:]

def Score_camel(population_size,burden_factor,death_rate,visibility,supply_init,endurance_init,min_temperature,max_temperature,n_estimator,criterion,max_feature):

    population_size = int(population_size)
    n_estimator = int(n_estimator)
    if criterion < 0.5:
        criterion = 'gini'
    else:
        criterion = 'entropy'
    if max_feature < 0.34:
        max_feature = 'sqrt'
    elif max_feature < 0.67:
        max_feature = 'log2'
    else:
        max_feature = None

    combo_str = f'{population_size}_{burden_factor}_{death_rate}_{visibility}_{supply_init}_{endurance_init}_{min_temperature}_{max_temperature}_{n_estimator}_{criterion}_{max_feature}'

    if combo_str in camel_dict:
        return camel_dict[combo_str]

    Algo = CamelAlgorithm()
    Algo.set_parameters(population_size=population_size,burden_factor=burden_factor,death_rate=death_rate,visibility=visibility,supply_init=supply_init,endurance_init=endurance_init,min_temperature=min_temperature,max_temperature=max_temperature,seed=SEED) 

    nia_mdl = NatureInspiredSearchCV(
        estimator=RandomForestClassifier(n_estimators=n_estimator,criterion=criterion,max_features=max_feature),         
        param_grid={},
        algorithm=Algo,
        runs=1,
    )
    nia_mdl.fit(X_train,y_train)
    val = nia_mdl.score(X_test,y_test)
    camel_dict[combo_str] = val 

    return val

CamelAlgorithm

SCORING FUNCTIONS (6)

FUNCTION LIST

In [75]:

NIA_Val_Dict = {0:'Cuckoo Search',1:'Fire Fly',2:'Bat',3:'Self Adaptive Bat',4:'Particle Swarm', 5:'Camel Algorithm'}

NIA_lst = [CucKoo_pbounds, FireFly_pbounds, BAT_pbounds, SABA_pbounds, PSA_pbounds, camel_bounds]

Save_dict_fxn = [Save_cuckoo_dict, Save_firefly_dict, Save_bat_dict, Save_sba_dict, Save_psa_dict, Save_camel_dict]

NIA_Score_fxn = [Score_Cuckoo, Score_Firefly, Score_Bat, Score_SBA, Score_PSA, Score_camel]

FUNCTION LIST

BASE CASE

In [80]:
if len(MyFinalResultDict) == 0:
  clf = RandomForestClassifier()
  clf.fit(X_train, y_train)
  val = clf.score(X_test,y_test)
  print(f'The default value is -> {val}')
  MyFinalResultDict['Base Result'] = val
  Save_Final_Dict()

BASE CASE

In [81]:
from bayes_opt import BayesianOptimization, UtilityFunction
import matplotlib.pyplot as plt


In [84]:
def Optimize_and_plot(bounds,score_fxn,Algo_name:str,Iters:int,Dict_save_fxn):
  
    print(f'Currently at {Algo_name}')

    optimizer = BayesianOptimization(f = None, pbounds = bounds, verbose = 2, random_state = SEED)
    utility = UtilityFunction(kind = "ucb", kappa = 1.96, xi = 0.01)

    for i in range(Iters):
        # Get optimizer to suggest a new parameter value to try.
        next_point = optimizer.suggest(utility)  
        # Evaluate the output of the black_box_function using the new parameter value.
        target = score_fxn(**next_point)
        try:
            # Update the optimizer with the evaluation results. This needs to be in try-except
            # to prevent repeat errors from occuring.
            optimizer.register(params = next_point, target = target)
        except:
            pass
    
    print(f'Best result: {optimizer.max["params"]}; f(x) = {optimizer.max["target"]}.')

    Dict_save_fxn()
    

    plt.plot(range(1, 1+len(optimizer.space.target)), optimizer.space.target, "-o")
    plt.grid(True)
    plt.xlabel("Iteration")
    plt.ylabel("Score")
    plt.legend()
    plt.savefig(f'{Algo_name}_{Iters}_Iterations.svg',format='svg')
    plt.show()

In [ ]:
for i in range(len(MyFinalResultDict)-1,len(NIA_lst)):
    Optimize_and_plot(bounds=NIA_lst[i],score_fxn=NIA_Score_fxn[i],Algo_name=NIA_Val_Dict[i],Dict_save_fxn=Save_dict_fxn[i],Iters=100)
    Save_Final_Dict()
    

Currently at Cuckoo Search
Fitting at most 1.0 candidates


function ClickConnect(){
               
console.log("Working"); 
document.querySelector("#comments > span").click() 
}
setInterval(ClickConnect,5000)